In [1]:
# Run Fine-Tune Llama2-7b on SE paired dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import textwrap

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "./results/final_merged_checkpoint/",
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer="meta-llama/Llama-2-7b-hf"
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
def qa(text: str, full = False):
    # textwrap.dedent gets rid of indenting at the start of each newline
    text = textwrap.dedent(f"""\
        Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

        ### Instruction:
        {text}

        ### Response:
        """)
    
    response = pipe(text, max_length=100, do_sample=True, top_k=50, top_p=0.95, temperature=1.0)
    response = response[0]['generated_text']
    response = response.split("### Response:\n")[1] if not full else response

    return response


In [5]:
print(qa("How do I apply?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
How do I apply?

### Response:
To apply to SUNY Brockport, you need to visit their website and follow the application process.


In [6]:
print(qa("Are SAT scores required at Brockport?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
Are SAT scores required at Brockport?

### Response:
For transfer students, SAT scores are no longer required for acceptance at Brockport.


In [20]:
print(qa("Does Brockport have a CS department?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
Does Brockport have a CS department?

### Response:
Yes, Brockport has a CS department.


In [8]:
print(qa("How can I report an incident that happened on campus?", full = True))

# NOT A VALID PHONE NUMBER!

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
How can I report an incident that happened on campus?

### Response:
You can report an incident that happened on campus by going through the Incident Reporting Center. This is accessible through the Incident Reporting Center website.


In [18]:
print(qa("whats the process of applying", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
whats the process of applying

### Response:
The process of applying to SUNY Brockport can be found on their website.


In [19]:
print(qa("what are some of the classes for CS?", full = True))

# Hallucination

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
what are some of the classes for CS?

### Response:
Some of the classes for CS are CS110 Computer Literacy, CS120 Introduction to Programming I, CS220 Introduction to Programming II, CS330
